# Evaluate Model Performance on the Test Set 

In [1]:
import numpy as np
import tensorflow as tf
from data import load_file, preProcessingScriber, encode_data, insert_target

from transformers import AutoTokenizer
from transformers import TFCamembertForMaskedLM

from datetime import datetime
import os
import json
import sys

In [2]:
### instantiate the tokenizer
tokenizer = AutoTokenizer.from_pretrained("jplu/tf-camembert-base", do_lower_case=True)

In [3]:
### path to weights
checkpointPath = "Models/20200428_180956/cp-004.ckpt"

## Hyper-parameters

In [4]:
n = 512

vocab_size = 32005
segment_size = 32
batch_size = 5
train_layer_ind = -2  # 0 for all model, -2 for only top layer
num_epochs = 2

hyperparameters = {
    'vocab_size': vocab_size,
    'segment_size': segment_size,
    'batch_size': batch_size
}

In [5]:
punctuation_enc = {
    'O': 0,
    'PERIOD': 1,
}

## Get Dataset

In [6]:
print('\nPRE-PROCESS AND PROCESS DATA')


# name of dataset with sentences
data_name = "Scriber"
fileName = 'Data' + data_name + '/' + 'extractValid_01.txt'


# from sentences to list of words+punctuation
data = load_file(preProcessingScriber(fileName))


X_, y_ = encode_data(data, tokenizer, punctuation_enc)
X = insert_target(X_, segment_size)
y = np.asarray(y_)


# get only an n of the data.
X = X[0:n]
y = y[0:n]


# one hot encode the labels
y = tf.one_hot(y, 4, dtype='int64').numpy()


dataset = tf.data.Dataset.from_tensor_slices((X, y)).batch(batch_size)


PRE-PROCESS AND PROCESS DATA


In [10]:
X.shape

(512, 32)

## Build the model

In [7]:
# build and compile model

bert_input = tf.keras.Input(shape=(segment_size), dtype='int32', name='bert_input')
x = TFCamembertForMaskedLM.from_pretrained("jplu/tf-camembert-base")(bert_input)[0]
x = tf.keras.layers.Reshape((segment_size*vocab_size,))(x)
dense_out = tf.keras.layers.Dense(4, activation='softmax')(x)

net = tf.keras.Model(bert_input, dense_out, name='network')

net.compile(optimizer='adam',
              loss=tf.losses.CategoricalCrossentropy(from_logits=False),
              metrics=[tf.keras.metrics.Recall(class_id=0, name='Rec_0'),
                       tf.keras.metrics.Precision(class_id=0, name='Prec_0'),
                       tf.keras.metrics.Recall(class_id=1, name='Rec_1'),
                       tf.keras.metrics.Precision(class_id=1, name='Prec_1'),
                      ])

In [8]:
# load the weights
net.load_weights(checkpointPath)

## Evaluate the model

In [9]:
evaluation = net.evaluate(dataset)

103/103 [==============================] - 51s 495ms/step - loss: 1.1718 - Rec_0: 0.9899 - Prec_0: 0.9663 - Rec_1: 0.1053 - Prec_1: 0.2857
